In [1]:
from keras.layers import Dense,Activation
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
from keras.utils import plot_model
import numpy as np

/home/drdh/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
fin=open("../data/alice_in_wonderland.txt","rb")
lines=[]
for line in fin:
    line=line.strip().lower()
    line=line.decode("ascii","ignore")
    if len(line)==0:
        continue
    lines.append(line)
fin.close()
text=" ".join(lines)

In [5]:
chars=set([c for c in text])
nb_chars=len(chars)
char2index=dict((c,i) for i,c in enumerate(chars))
index2char=dict((i,c) for i,c in enumerate(chars))

In [13]:
SEQLEN=10
STEP=1

input_chars=[]
label_chars=[]
for i in range(0,len(text)-SEQLEN,STEP):
    input_chars.append(text[i:i+SEQLEN])
    label_chars.append(text[i+SEQLEN])

In [9]:
X=np.zeros((len(input_chars),SEQLEN,nb_chars),dtype=np.bool)
y=np.zeros((len(input_chars),nb_chars),dtype=np.bool)
for i,input_char in enumerate(input_chars):
    for j,ch in enumerate(input_char):
        X[i,j,char2index[ch]]=1
    y[i,char2index[label_chars[i]]]=1

In [10]:
HIDDEN_SIZE=128
BATCH_SIZE=128
NUM_ITERATIONS=25
NUM_EPOCHS_PER_ITERATION=1
NUM_PREDS_PER_EPOCH=100

model=Sequential()
model.add(SimpleRNN(HIDDEN_SIZE,return_sequences=False,input_shape=(SEQLEN,nb_chars),unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy",optimizer="rmsprop")

In [11]:
plot_model(model,to_file="SimpleRNN.png",show_shapes=True)

In [12]:
for iteration in range(NUM_ITERATIONS):
    print("=" *50)
    print("Iteration #: %d"%(iteration))
    model.fit(X,y,batch_size=BATCH_SIZE,epochs=NUM_EPOCHS_PER_ITERATION)
    
    test_idx=np.random.randint(len(input_chars))
    test_chars=input_chars[test_idx]
    print("Gerationg from seed: %s"%(test_chars))
    print(test_chars,end="")
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest=np.zeros((1,SEQLEN,nb_chars))
        for i,ch in enumerate(test_chars):
            Xtest[0,i,char2index[ch]]=1
        pred=model.predict(Xtest,verbose=0)[0]
        ypred=index2char[np.argmax(pred)]
        print(ypred,end="")
        test_chars=test_chars[1:]+ypred
    print('\n')

Iteration #: 0
Epoch 1/1
158773/158773 [==============================] - 17s 106us/step - loss: 2.3400
Gerationg from seed:  it yet, s
 it yet, she said the said the said the said the said the said the said the said the said the said the said th

Iteration #: 1
Epoch 1/1
158773/158773 [==============================] - 17s 108us/step - loss: 2.0480
Gerationg from seed: hat she be
hat she began the gate the grome the said the gate the grome the said the gate the grome the said the gate the

Iteration #: 2
Epoch 1/1
158773/158773 [==============================] - 17s 107us/step - loss: 1.9449
Gerationg from seed: ppose, by 
ppose, by the dont the was the fore the project gutenberg-t alice said the dound the dont the was the fore the

Iteration #: 3
Epoch 1/1
158773/158773 [==============================] - 16s 104us/step - loss: 1.8643
Gerationg from seed: esent--at 
esent--at whe said the dore the said the dore the said the dore the said the dore the said the dore the said t

Iteratio